# Long text summarization using LCEL chains on Langchain with Bedrock APIs

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Overview
When we work with large documents, we can face some challenges as the input text might not fit into the model context length, or the model hallucinates with large documents, or, out of memory errors, etc.

To solve those problems, we are going to show a solution that is based on the concept of chunking and chaining prompts. This solution is leveraging [LangChain](https://python.langchain.com/docs/get_started/introduction.html) which is a popular framework for developing applications powered by language models.

In this architecture:

1. A large document (or a giant file appending small ones) is loaded
1. Langchain utility is used to split it into multiple smaller chunks (chunking)
1. First chunk is sent to the model; Model returns the corresponding summary
1. Langchain gets next chunk and appends it to the returned summary and sends the combined text as a new request to the model; the process repeats until all chunks are processed
1. In the end, you have final summary based on entire content

### Use case
This approach can be used to summarize call transcripts, meetings transcripts, books, articles, blog posts, and other relevant content.

### Imports

In [1]:
import os
import sys
from langchain_aws import ChatBedrockConverse
from IPython.core.display import HTML
from IPython.display import display_markdown, Markdown
import boto3

HTML("<script>Jupyter.notebook.kernel.restart()</script>")

module_path = ".."
sys.path.append(os.path.abspath(module_path))

boto3_bedrock = boto3.client('bedrock-runtime')

textgen_llm = ChatBedrockConverse(
    model_id="us.amazon.nova-micro-v1:0",
    client=boto3_bedrock,
    max_tokens=None,
    temperature=0.5
)


### Load shareholder letter

We will be following a process similar to lab 02 in this summarization section. First, let us load the 2022 Amazon shareholder letter

In [2]:
shareholder_letter = "./letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()

In [3]:
len(letter.split(' '))

5084

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=8096, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [5]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import XMLOutputParser
from langchain.schema.output_parser import StrOutputParser


xml_parser = XMLOutputParser(tags=['insight'])
str_parser = StrOutputParser()

prompt = PromptTemplate(
    template="""
    
    Human:
    {instructions} : \"{document}\"
    Format help: {format_instructions}.
    Assistant:""",
    input_variables=["instructions","document"],
    partial_variables={"format_instructions": xml_parser.get_format_instructions()},
)

insight_chain = prompt | textgen_llm | StrOutputParser()

In [6]:
print(f"Number of Documents {len(docs)}")

Number of Documents 5


# Option 1. Manually process insights, then summarize

In [7]:
%%time
insights=[]
for i in range(len(docs)):
    insights.append(
        insight_chain.invoke({
        "instructions":"Provide Key insights from the following text",
        "document": {docs[i].page_content}
    }))

CPU times: user 40.7 ms, sys: 1.26 ms, total: 41.9 ms
Wall time: 7.72 s


In [8]:
str_parser = StrOutputParser()

prompt = PromptTemplate(
    template="""
    
    Human:
    {instructions} : \"{document}\"
    Assistant:""",
    input_variables=["instructions","document"]
)

summary_chain = prompt | textgen_llm | StrOutputParser()

In [9]:
%%time
display_markdown(Markdown(summary_chain.invoke({
        "instructions":"You will be provided with multiple sets of insights. Compile and summarize these insights and provide key takeaways in one concise paragraph. Do not use the original xml tags. Just provide a paragraph with your compiled insights.",
        "document": {'\n'.join(insights)}
    })))

Despite facing macroeconomic challenges and internal operating difficulties in 2022, Amazon managed to grow its consumer business significantly, with revenue increasing from $245B in 2019 to $434B in 2022, driven by expansion in fulfillment centers and a robust transportation network. The company's AWS business continued to thrive with a 29% YoY growth rate and a revenue run rate of $85B, despite macroeconomic conditions prompting companies to be more cautious in spending. Amazon's Advertising business has also shown significant growth, leveraging machine learning algorithms to tailor sponsored products. Amazon continues to innovate across various sectors, including the introduction of 'Buy with Prime' to help third-party brands and sellers drive conversions, and the launch of Amazon Pharmacy and the acquisition of One Medical to enhance healthcare services. Amazon's strategic investments in machine learning and Generative AI, particularly in Large Language Models, are poised to drive future growth across all business areas, with a strong belief that Amazon's best days are ahead.

CPU times: user 6.8 ms, sys: 0 ns, total: 6.8 ms
Wall time: 1.18 s


# Option 2. Use Map reduce pattern on Langchain

In [10]:
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=textgen_llm, chain_type="map_reduce", verbose=False, token_max=1024)

In [11]:
%%time
display_markdown(Markdown(summary_chain.invoke(docs)['output_text']))

In his second annual shareholder letter, Amazon's CEO expresses optimism about the company's future despite macroeconomic challenges and internal difficulties in 2022. He highlights Amazon's growth in retail, cloud services (AWS), and advertising, emphasizing long-term investments and strategic adjustments. AWS continues to grow with an $85B revenue run rate and has launched over 3,300 new features, including advanced machine learning chips. Amazon has optimized its fulfillment network for efficiency and cost-effectiveness, and its Advertising business is thriving with tailored ads based on machine learning. The company is expanding into new markets, sectors like healthcare and broadband, and is investing heavily in Generative AI and Large Language Models to drive innovation. Amazon remains confident in its ability to grow and innovate, driven by customer focus and relentless innovation.

CPU times: user 3.15 s, sys: 351 ms, total: 3.5 s
Wall time: 10.6 s


# Reference - Read Full Shareholder Letter
Optionally here please run the next cell to view the Shareholder letter. Cross reference with the outputs of options 1 and 2 to gage the effectiveness of the summerization prompts


In [12]:
print(letter)

As I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and energized by what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic years in recent memory, and with some of our own operating challenges to boot, we still found a way to grow demand (on top of the unprecedented growth we experienced in the first half of the pandemic). We innovated in our largest businesses to meaningfully improve customer experience short and long term. And, we made important adjustments in our investment decisions and the way in which we’ll invent moving forward, while still preserving the long-term investments that we believe can change the future of Amazon for customers, shareholders, and employees.

While there were an unusual number of simultaneous challenges this past year, the reality is that if you operate in large, dynamic, global market segments with many capable and well-funded competitors (the conditions in which Amazon operates all of it